# Structural learning

In this notebook you will understand the basic concepts of structural learning of discrete Bayesian networks.

Remember that a Bayesian network is based on a Directed Acyclic Graph (DAG). It is precisely this graph structure  what we want to learn from data in this case.

Let's start loading the necessary libraries.

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import copy

from pgmpy.models import BayesianModel
from pgmpy.sampling import BayesianModelSampling
from pgmpy.utils import get_example_model

## Generating data from a known BN

Let's play with a real BN. Thus, we can test if we are able to recover the real structure.

To do so, we will use again the <a href="https://www.bnlearn.com/bnrepository/discrete-small.html#survey">survey</a> BN.

<img src="https://www.bnlearn.com/bnrepository/survey/survey.png" width="300" />
We will learn from the generated data and compare with the real structure. We will use pgmpy to do so:

In [ ]:
np.random.seed(7)
gen_model = get_example_model('survey')
nx.draw(gen_model, with_labels=True, arrowsize=30, node_size=800, alpha=0.3, font_weight='bold')
plt.show()

n_samples = 1000

samples = BayesianModelSampling(gen_model).forward_sample(size=n_samples)
samples.head()

## Mutual information
To learn the structure we need to find dependencies between variables to know where introduce an edge.

We can use, to this end, mutual information which, as we explained at class, is related to maximum likelihood estimators in its basic form. Let's calculate it as:
$$MI(X,Y)=\sum_{x_i,y_j} \frac{N_{x_i,y_j}}{N} \log \frac{N\cdot N_{x_i,y_j}}{N_{x_i}\cdot N_{y_j}} $$

In [ ]:
from sklearn.metrics.cluster import contingency_matrix
def mutual_information(x,y,smooth=0.1): 
    # first, we calculate the contingency table between both variables
    contingency = contingency_matrix(x,y)+smooth # we use smoothing to avoid problems with 0 probabilities
    # we obtain per-variable counts
    N = np.sum(contingency)
    Ni = contingency.sum(axis=1, keepdims=True)
    Nj = #### YOUR CODE HERE ####

    inner = np.log(contingency / Ni / Nj * N)
    outer = contingency / N
    return np.sum(inner*outer)

We can calculate it, now, for the second and third variables as:

In [ ]:
mutual_information(samples.values[:,1],samples.values[:,2])

The conditional mutual information is defined similarly, but with estimates of the conditional distributions on a third variable:
$$CMI(X,Y;Z)=\sum_{z_k} \sum_{x_i,y_j} \frac{N_{x_i,y_j,z_k}}{N} \log \frac{N_{z_k}\cdot N_{x_i,y_j}}{N_{x_i,z_k}\cdot N_{y_j,z_k}} $$

In [ ]:
def cond_mutual_information(x,y,cond_z):
    z_vals = np.unique(cond_z)
    N = len(x)
    res = 0
    for i_z, z in enumerate(z_vals):
        ind_zs = np.where(cond_z == z)[0]
        contingency = contingency_matrix(x[ind_zs],y[ind_zs])
        Nz = np.sum(contingency)
        Niz = contingency.sum(axis=1, keepdims=True)
        Njz = #### YOUR CODE HERE ####

        inner = np.log(contingency / Niz / Njz * Nz)
        outer = contingency / N
        res += np.sum(inner*outer)
    return res

Similar to the previous case, if we condition on the fourth variable, we obtain:

In [ ]:
cond_mutual_information(samples.values[:,1],samples.values[:,2],samples.values[:,3])

We might need to calculate the mutual information between sets of variables. To do so, we transform each set of variables into a new single variable as the cartesian product of the original variables. Then, we can calculate MI as in the regular case:

In [ ]:
def mutual_information_sets(x,y,cards_x,cards_y):
    if x.shape[1] > 1:
        cols=[]
        for j in np.arange(x.shape[1]):
            cols.append(np.unique(x[:,j], return_inverse=True)[1])
        x = np.ravel_multi_index(cols, cards_x)
    if y.shape[1] > 1:
        cols=[]
        for j in np.arange(y.shape[1]):
            cols.append(np.unique(y[:,j], return_inverse=True)[1])
        y = np.ravel_multi_index(cols, cards_y)
    return mutual_information(x,y)

In this case, to be able to calculate the cartesian product, we need to know the cardinality of the involved variables. This is how we calculate it:

In [ ]:
# calculate the cardinalities
cards = [len(np.unique(samples.values[:,j])) for j in np.arange(samples.shape[1])]
# calculate MI(X_1,X_2 ; X_3,X_4)
mutual_information_sets(samples.values[:100,:2].astype(str),
                        samples.values[:100,2:4].astype(str),
                        cards[:2],cards[2:4])

## Learning a BN tree
Now we have all we need to start learning a network. We can start by calculating the mutual information between all the pairs of variables, if that is feasible. We are going to order them by decreasing value too:

In [ ]:
edges = {}
for i in np.arange(samples.shape[1]-1):
    for j in np.arange(i+1,samples.shape[1]):
        edges[(i,j)] = mutual_information(samples.values[:,i],samples.values[:,j])
sorted_edges = dict(sorted(edges.items(), key=lambda item: item[1], reverse=True))
sorted_edges

If we create a network trying to reach all the nodes (so that none is disconnected) and avoiding loops, we could do something like this:

In [ ]:
G=nx.Graph()
G.add_nodes_from(np.arange(samples.values.shape[1]))

n = samples.values.shape[1] - 1
for k in sorted_edges:
    if not nx.has_path(G,k[0],k[1]):
        G.add_edge(k[0],k[1])
        n-=1
        if (n == 0):
            break;

Note that we introduce only $n-1$ edges (where $n$ is the number of nodes). Try it yourself, it is the maximum number if we don't want cycles.

Let's plot the graph that we just created:

In [ ]:
mapping = {k:v for k,v in zip(np.arange(samples.values.shape[1]),np.array(samples.columns,dtype=str))}
Gp = nx.relabel_nodes(G, mapping)
nx.draw_circular(Gp, with_labels=True, arrowsize=30, node_size=800, alpha=0.3, font_weight='bold')
plt.show()

Look at it! It's a **tree**! You probably have already noticed that we are just applying <a href="https://en.wikipedia.org/wiki/Kruskal%27s_algorithm">Kruskal's algorithm</a> for finding maximum spanning trees. 

The <a href="https://en.wikipedia.org/wiki/Chow%E2%80%93Liu_tree">Chow-Liu algorithm</a> to learn a BN tree might be understood as using Kruskal's method and, then, directing all the edges following this procedure:

- 1) Select a node as a root
- 2) Direct all the edges related with it as out-edges
- 3) Identify all the reached nodes, and follow the same procedure as in Step 2 for all of them.

In [ ]:
H=nx.DiGraph()
H.add_nodes_from(np.arange(samples.values.shape[1]))

root_node = 1 # we choose this, as a random choice
# we use these lists to identify the set of currently processing nodes and the previously processed ones
next_list = []
act_list = [root_node]
prev_list = []
while len(act_list)>0:
    for n in act_list:
        nbs_n = nx.neighbors(G,n) # for each processing node, find all neighbors
        for nei in nbs_n:
            if nei not in prev_list: # for each neighbor, if it is not a previously processed node, add directed edge
                H.add_edge(n,nei)
                next_list.append(nei)
        prev_list.append(n)
    act_list = next_list
    next_list=[]

Now, we can draw it again:

In [ ]:
mapping = {k:v for k,v in zip(np.arange(samples.values.shape[1]),np.array(samples.columns,dtype=str))}
Hp = nx.relabel_nodes(H, mapping)
nx.draw_circular(Hp, with_labels=True, arrowsize=30, node_size=800, alpha=0.3, font_weight='bold')
plt.show()

We already have our DAG structure!! We would only need to learn the parameters, after this.

### Using pgmpy for learning a tree
We can use pgmpy to do all this. To use Chow-Liu's algorithm, we need to do:

In [ ]:
from pgmpy.estimators import TreeSearch

# let's just search for a tree. Remember that in this case, the root is not that important
est = TreeSearch(samples, root_node="A")
# estimate the DAG using Chow-Liu's algorithm
cl_dag = est.estimate(estimator_type="chow-liu")

nx.draw_circular(cl_dag, with_labels=True, arrowsize=30, node_size=800, alpha=0.3, font_weight='bold')
plt.show()

We now can learn the parameters associated with this structure similarly as we did in the previous notebook:

In [ ]:
from pgmpy.estimators import MaximumLikelihoodEstimator

# we will stick to Maximum Likelihood (but you know about the existence of Bayesian estimators...)
cl_model = BayesianModel(cl_dag.edges())
cl_model.fit(data=samples, estimator=MaximumLikelihoodEstimator)
cl_model.get_cpds()

We would like to score somehow the structure that we just obtained. Sticking to the same criteria, we can calculate the likelihood of the model given the sample. 

We need to calculate the probability of the samples (which, surprisingly, cannot be done in pgmpy!!) given that specific model.

We are going to use log probabilities to avoid numerical issues!

In [ ]:
def log_probability_of_samples(G, data):
    res = np.zeros(data.shape[0])
    lcpds = G.get_cpds()
    for i in np.arange(len(res)):
        smp_d = dict(data.iloc[i,:])
        for cpd in lcpds:
            evidence = { k: smp_d[k] for k in cpd.scope() }
            prob = cpd.get_value(**evidence)
            #### YOUR CODE HERE ####
    return res

def log_likelihood(G, data):
    probs = log_probability_of_samples(G, data)
    probs = probs[np.isfinite(probs)] # sanity check!
    return np.sum(probs)

def likelihood(G, data):
    return np.exp(log_likelihood(G, data))

Let's compare the likelihood of our model with that of the original one (the one we used to generate the data):

In [ ]:
print(log_likelihood(gen_model, samples))
print(log_likelihood(cl_model, samples))

You can observe that the likelihood of the learnt structure is larger than that of the generative model!! Remember what we mention at class about the tendency of MLE to overfitting??

## Learning a general DAG
Now, we want to learn an unrestricted DAG (not a tree, as before). We will use a Hill-Climbing greedy (local search) method.

The first thing that we need to implement, to take full advantage of the process, is a score that decomposes locally within nodes/CPDs. We use the likelihood, which reduces to the MI of each node (the main variable with the parent variables):

In [ ]:
def local_score(dag, act_score, data, vars_to_update):
    new_score = copy.deepcopy(act_score)
    for var in vars_to_update:
        edges_from_par = dag.in_edges(var)
        evid_var = [e for e,_ in edges_from_par]
        if len(evid_var) > 0:
            new_score[var] = mutual_information_sets(data[[var]].values.astype(str),
                                                     data[evid_var].values.astype(str),
                                                     [cards[var]],[cards[k] for k in evid_var])
    return new_score

Then, we implement a few useful functions:
- one that finds all the possible local changes (swap, removal, addition) for a given structure
- another one that implements such a change into a given DAG
- a function that calculates the score of a possible change

In [ ]:
def rev(arc):
    return (arc[1],arc[0])

def find_all_changes(dag, anc_ordering):
    changes = list(dag.edges()) # we can remove these edges
    
    # we consider a new potential parent all the previous nodes in the ordering
    for v in anc_ordering:
        for prev_v in anc_ordering:
            if prev_v == v:
                continue;
            if (prev_v, v) in dag.edges(): # if the edge is already in the DAG
                continue;
            if nx.has_path(dag, v, prev_v): # if the new edge would introduce a loop
                continue;
            changes.append((prev_v, v))
    
    for edge in dag.edges():
        prov_dag = dag.copy()
        prov_dag.remove_edge(edge[0], edge[1])
        rev_edge = rev(edge)
        if not nx.has_path(prov_dag,edge[0], edge[1]):
            changes.append(rev(edge))
    return changes

def apply_change(dag, change):
    if change in dag.edges():
        # remove
        dag.remove_edge(change[0], change[1])
    elif rev(change) in dag.edges():
        revedge = rev(change)
        dag.remove_edge(revedge[0], revedge[1])
        dag.add_edge(change[0],change[1])
    else:
        # add
        dag.add_edge(change[0],change[1])

def score_of_possible_change(dag, act_score, data, change):
    prov_dag = dag.copy()
    if change in dag.edges():
        # remove
        prov_dag.remove_edge(change[0], change[1])
        return local_score(prov_dag, act_score, data, [change[1]])
    elif rev(change) in dag.edges():
        revedge = rev(change)
        prov_dag.remove_edge(revedge[0], revedge[1])
        prov_dag.add_edge(change[0],change[1])
        return local_score(prov_dag, act_score, data, [change[0], change[1]])
    else:
        # add
        prov_dag.add_edge(change[0],change[1])
        return local_score(prov_dag, act_score, data, [change[1]])

Finally, the algorithm is just a greedy method that selects, in each step, the best possible change, until no change can improve the current structure:

In [ ]:
def hill_climbing(dag, anc_ordering, data):
    lh_increases = True
    prev_score = 0
    prev_dcmp_score = {c:0 for c in data.columns}
    while lh_increases:
        all_posible_changes = find_all_changes(dag, anc_ordering)
        act_best_score = 0
        act_dcmp_score = None
        act_change = None
        for change in all_posible_changes:
            local_lh = score_of_possible_change(dag, prev_dcmp_score, data, change)
            ch_sc = np.sum(list(local_lh.values())) #
            print(change,ch_sc)
            if #### YOUR CODE HERE ####: 
                act_best_score = ch_sc
                act_dcmp_score = local_lh
                act_change = change
        if act_best_score > prev_score:
            prev_score = act_best_score
            prev_dcmp_score = act_dcmp_score
            apply_change(dag, act_change)
        else:
            lh_increases = False
    return dag

We have now all we need!! Let's try it. 

We start by creating an empty BN (no edges), which will be our initial point (it could be another one! E.g., the tree learnt with Chow-Liu).

In [ ]:
initial_bn = BayesianModel() 
initial_bn.add_nodes_from(samples.columns)
initial_bn.fit(data=samples, estimator=MaximumLikelihoodEstimator)
ordering = list(samples.columns) # In this algorithm, we really don't care about the ordering

We are going to need the cardinality of the features:

In [ ]:
cards = {c:len(np.unique(samples[c].values)) for c in samples.columns}
cards

And now we just run our local search algorithm and, for the obtained DAG, we learn the parameters:

In [ ]:
final_bn = hill_climbing(initial_bn, ordering, samples)
final_bn.fit(data=samples, estimator=MaximumLikelihoodEstimator)
final_bn.get_cpds()

Let's see the likelihood of this model, and how does it look:

In [ ]:
print(log_likelihood(final_bn, samples))
nx.draw_circular(final_bn, with_labels=True, arrowsize=30, node_size=800, alpha=0.3, font_weight='bold')
plt.show()

It resembles so much the complete graph, isn't it?? Remember that tendency of MLE to overfeating?

### Learning a general DAG with pgmpy

We now are going to do this using pgmpy methods but, let's take the opportunity to use a more robust score: <a href="https://en.wikipedia.org/wiki/Bayesian_information_criterion">BIC</a>:

In [ ]:
from pgmpy.estimators import BicScore,HillClimbSearch

scoring_method = BicScore(data=samples)
est = HillClimbSearch(data=samples)
hc_dag = est.estimate(scoring_method=scoring_method, max_indegree=4, max_iter=int(1e4))

# we will stick to Maximum Likelihood (but you know the existence of Bayesian estimators...)
hc_model = BayesianModel(hc_dag.edges())
hc_model.fit(data=samples, estimator=MaximumLikelihoodEstimator)
hc_model.get_cpds()

Let's see the likelihood of this model, and how does it look:

In [ ]:
print(log_likelihood(hc_model, samples))
nx.draw_circular(hc_model, with_labels=True, arrowsize=30, node_size=800, alpha=0.3, font_weight='bold')
plt.show()